In [1]:
import numpy as np
import pandas as pd

In [3]:
# ПРИХИ

# Загружаем таблицу с приходами по всем ТГ
cs_prih = pd.read_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\Отдел закупки\ЛИЧНЫЕ\Семен\заказы\4001.xlsx',
    usecols=[0,4,9,11,12,15],names=['Производитель','Тип 1','Номер','Приходы_4001','ТПГ','ОПТ'])

# Убираем "итог", оптовый товар и нулевые приходы

cs_prih.drop(index=len(cs_prih.Производитель.tolist())-1,inplace=True)

cs_prih = cs_prih[(cs_prih['ОПТ'] == 'Нет') & (cs_prih['Приходы_4001'] != 0)]
del cs_prih['ОПТ']

# Создаем 2 датафрейма с приходами ИФ и БК
prih_if = cs_prih[cs_prih['ТПГ']< 1499]

prih_bk = cs_prih[(cs_prih['ТПГ'] > 1499) &
                  (cs_prih['Тип 1'] != 'КОЛЬЦО ОБРУЧАЛЬНОЕ')& 
                  (cs_prih['Тип 1'] != 'СЕРЬГИ-КОНГО')]

del prih_if['ТПГ'], prih_bk['ТПГ']


In [2]:
# Дефицит ИФ

import os

def getfiles(dirpath):
    a = [s for s in os.listdir(dirpath) if '~$' not in s]
    a.sort(key=lambda s: os.path.getmtime(os.path.join(dirpath, s)))
    return os.path.join(dirpath, a[-1])

mydir = r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\Отдел закупки\ЛИЧНЫЕ\Карпова Ксения\ИФ\Дефицит'

deficit_df_if = pd.read_excel(getfiles(mydir),sheet_name='ЛЕНГОЛД', 
                              skiprows=2, usecols=[1,19],names=['tpg','deficit'],nrows=102)

deficit_dict_if = deficit_df_if.set_index('tpg').T.to_dict('records')[0]

In [4]:
#КОЛЬЦА ИФ

# Загружаем две таблицы остатки и продажи КОЛЬЦА
balance_rings = pd.read_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\Отдел закупки\ЛИЧНЫЕ\Семен\заказы\ифмарт1.xlsx',
    sheet_name='ост_к',skiprows=9) #использовать sheet_name='ост' для неколец

sales = pd.read_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\Отдел закупки\ЛИЧНЫЕ\Семен\заказы\ифмарт1.xlsx',
    sheet_name='прод',skiprows=12,usecols=list(range(2,9)))

# Фильтр приходов по кольцам
prih_if_rings = prih_if[
    (prih_if['Тип 1'] == 'КОЛЬЦО')|
    (prih_if['Тип 1'] == 'КОЛЬЦО ОБРУЧАЛЬНОЕ')|
    (prih_if['Тип 1'] == 'КОЛЬЦО ПЕЧАТКА')
]

# Притягиваем к остаткам продажи, потом добавляем к этому приходы через 'outer'

merged = pd.merge(prih_if_rings,(pd.merge(balance_rings,sales,on='Номер',how = 'left',suffixes=('_ос', '_пр'))),how = 'outer')

# соединяем приходы, остатки и продажи
#merged = pd.merge(prih_if,merged,how = 'outer')

# Добавляем в финальную таблицу недостающую инфу по карточкам
items = pd.read_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\Отдел закупки\ЛИЧНЫЕ\Семен\заказы\ифмарт1.xlsx',
    sheet_name='карточки',skiprows=6)
                  
del items['Описание 3']

final = pd.merge(merged,items,on='Номер',how = 'left')

# Удаляем опт по группе наценки, ИМ по артикулу
final = final[~(final['Группа наценки'].str.contains('опт',case=False,na=False))]#na=False
final = final[~(final['Артикул товара'].str.contains('#ИМ',case=True,na=False))]#na=False

#Суммируем приходы, продажи остатки чтобы и убираем товы с нулями
#final['сумма']=final.groupby(by='Артикул товара')[col_names[5:7]].sum(axis=1)

# Выстраиваем столбцы в нужном порядке и сохраняем
col_names = final.columns.tolist()

columns = (['Артикул товара', 'Тип 1', 'Производитель', 'Товарная подгруппа'] +
           ['Ценовая корзина', 'Номер', 'Описание', 'Дизайн', 'Размер','Количество камней'] +
           col_names[10:16] + col_names[4:10] +
           ['Приходы_4001', 'Средний вес изделия'] +
           ['Тип 3','Вставка камней', 'Фото изделия', 'Тип изготовления'])

final = final[columns]
#final=final.set_index('Артикул товара',drop=True, inplace=True)

final.to_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\Отдел закупки\ЛИЧНЫЕ\Семен\заказы\иф_кольца.xlsx')

In [9]:
#НЕКОЛЬЦА ИФ

# Загружаем две таблицы остатки и продажи НЕКОЛЬЦА
balance = pd.read_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\Отдел закупки\ЛИЧНЫЕ\Семен\заказы\ифмарт1.xlsx',
    sheet_name='ост',skiprows=9) #использовать sheet_name='ост' для неколец

sales = pd.read_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\Отдел закупки\ЛИЧНЫЕ\Семен\заказы\ифмарт1.xlsx',
    sheet_name='прод',skiprows=12,usecols=list(range(2,9)))

# фильтр приходов по кольцам
prih_if_norings = prih_if[
    (prih_if['Тип 1'] != 'КОЛЬЦО')&
    (prih_if['Тип 1'] != 'КОЛЬЦО ОБРУЧАЛЬНОЕ')&
    (prih_if['Тип 1'] != 'КОЛЬЦО ПЕЧАТКА')
]

# Притягиваем к остаткам продажи, потом добавляем к этому приходы через 'outer'
merged = pd.merge(prih_if_norings,(pd.merge(balance,sales,on='Номер',how = 'left',suffixes=('_ос', '_пр'))),how = 'outer')

# добавляем в финальную таблицу недостающую инфу по карточкам
items = pd.read_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\Отдел закупки\ЛИЧНЫЕ\Семен\заказы\ифмарт1.xlsx',
    sheet_name='карточки',skiprows=6)
del items['Размер']

final = pd.merge(merged,items,on='Номер',how = 'left')

# Удаляем опт, ИМ
final=final[~(final['Группа наценки'].str.contains('опт',case=False,na=False))]
final=final[~(final['Артикул товара'].str.contains('#ИМ',case=True,na=False))]

# Добавляем дефицит по тпг
final['Дефицит по тпг'] = final['Товарная подгруппа'].map(deficit_dict_if)

# Выстраиваем столбцы в нужном порядке
col_names = final.columns.tolist()

columns = (['Артикул товара', 'Тип 1', 'Производитель', 'Товарная подгруппа'] +
           ['Ценовая корзина', 'Номер', 'Описание', 'Дизайн', 'Описание 3','Количество камней'] +
           col_names[10:16] + col_names[4:10] +
           ['Приходы_4001', 'Средний вес изделия'] +
           ['Дефицит по тпг', 'Тип 3','Вставка камней', 'Фото изделия', 'Тип изготовления'])

final = final[columns]

#Суммируем приходы, продажи остатки чтобы и убираем товы с нулями
#final['сумма'] = final[col_names[3:18]].sum(axis=1)
#final=final[final['сумма'] >0]
#del final['сумма']

#Удаляем буквы и зодиаки
final = final[(final['Дизайн'] !='ИФ БУКВЫ') & (final['Дизайн'] !='ИФ ЗОДИАК')]

final.to_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\Отдел закупки\ЛИЧНЫЕ\Семен\заказы\иф_некольца.xlsx')


In [13]:
#БК МОЖНО ОТКЛЮЧИТЬ НЕ нужный тип1 в конце

# Загружаем две таблицы остатки 4001 и 4093 и продажи
balance = pd.read_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\Отдел закупки\ЛИЧНЫЕ\Семен\заказы\бк.xlsx',
    sheet_name='ост',skiprows=9)

sales = pd.read_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\Отдел закупки\ЛИЧНЫЕ\Семен\заказы\бк.xlsx',
    sheet_name='прод',skiprows=12, usecols=list(range(2,9)))

# Притягиваем к остаткам продажи, потом добавляем к этому приходы через 'outer'
merged = pd.merge(prih_bk,(pd.merge(balance,sales,on='Номер',how = 'left',suffixes=('_ос', '_пр'))),how = 'outer')

# Добавляем в финальную таблицу недостающую инфу по карточкам
items = pd.read_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\Отдел закупки\ЛИЧНЫЕ\Семен\заказы\бк.xlsx',
    sheet_name='карточки',skiprows=6)

final = pd.merge(merged,items,on='Номер',how = 'left')

# Удаляем опт, ИМ
final = final[~(final['Группа наценки'].str.contains('опт',case=False,na=False))]#na=False
final = final[~(final['Артикул товара'].str.contains('#ИМ',case=True,na=False))]#na=False

# Создаем словарь с дефицитом по тпг и добавляем данные в таблицу
deficit_df_bk = pd.read_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\Отдел закупки\ЛИЧНЫЕ\Ксю\Дефицит по ТПГ 2019.xlsx',
    sheet_name='Дефицит', skiprows=2, usecols=[0,18],names=['tpg','deficit'],nrows=93)

deficit_df_bk['tpg'] = deficit_df_bk.tpg.astype('float64')
deficit_dict_bk = dict(zip(deficit_df_bk.tpg, deficit_df_bk.deficit))
final['дефицит по тпг'] = final['Товарная подгруппа'].map(deficit_dict_bk)


# Выстраиваем столбцы в нужном порядке
col_names = final.columns.tolist()

columns = (['Артикул товара', 'Тип 1', 'Производитель', 'Товарная подгруппа'] +
           ['Ценовая корзина', 'Номер', 'Описание', 'Дизайн', 'Размер', 'Описание 3'] +
           col_names[10:16] + col_names[4:10] +
           ['Приходы_4001','Средний вес изделия'] +
           ['дефицит по тпг', 'Тип 3', 'Фото изделия', 'Тип изготовления'])

final = final[columns]


#final=final.set_index('Артикул товара',drop=True, inplace=True)

# Фильтруем по кольцам убираем описание 3 и суммируем приходы 4001+4093 в "приходы". 
final_ring = final[final['Тип 1'].str.contains('КОЛЬЦО')].drop(['Описание 3'], axis=1)

# Фильтруем по некольцам и убираем размер и буквы с зодиаками
final_other = final[~final['Тип 1'].str.contains('КОЛЬЦО')].drop(['Размер'], axis=1)

final_other = final_other[~((final_other['Тип 1']== 'ПОДВЕС ДЕКОРАТИВНЫЙ') & 
                            ((final_other['Дизайн'] == 'ИФ ДВУСПЛАВ ДЕКОР') | 
                            (final_other['Дизайн'] =='ИФ БУКВЫ') | 
                            (final_other['Дизайн'] =='ИФ ЗОДИАК')))]

# сохраняем кольца и некольца на отдельные вкладки
with pd.ExcelWriter (
     r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\Отдел закупки\ЛИЧНЫЕ\Семен\заказы\заказы_бк.xlsx'
) as writer:
    final_ring.to_excel(writer, sheet_name = 'кольца')
    final_other.to_excel(writer, sheet_name = 'некольца')

'\\\\gold585.int\\uk\\Общее хранилище файлов\\Коммерческий департамент\\Отдел закупки\\ЛИЧНЫЕ\\Карпова Ксения\\ИФ\\ДефицитДефицит по ТПГ 05.08.19 Внутрихолдинг.xlsx'

'Дефицит по ТПГ 05.08.19 Внутрихолдинг.xlsx'